<a href="https://colab.research.google.com/github/karank85/speech-recognition/blob/main/benchmark-DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [1]:
import os

import numpy as np
from numpy import ndarray
import pandas as pd
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import librosa

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa.display
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

from IPython.display import Audio

!pip install transformers datasets evaluate jiwer accelerate

from datasets import Dataset, DatasetDict,Features, Array3D, Value, concatenate_datasets

from transformers import AutoProcessor, AutoModelForCTC, TrainingArguments, Trainer

import evaluate

import numpy as np

import glob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## Prepare Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Assumptions:
# - The transcription file is located in the same directory as the audio files.
class AudioDataset:
  """
  Class for loading and storing audio data.
  """

  def __init__(self):
    self.df = pd.DataFrame(columns=['id', 'path', 'transcription'])

  def load_transcriptions(self, directory_path: str) -> bool:
    """
    Load all transcriptions from a given directory, including subdirectories.
    Returns False if no transcription files were found, or if any failed to load.
    """
    sound_names = glob.glob(
        f"{directory_path}/**/*.wav",
        recursive=True
    )

    if len(sound_names) == 0:
      return False

    for path in sound_names:
      if not self.load_transcription_file(path):
        return False

    return True



  def load_transcription_file(self, file_path: str) -> bool:
    """
    Parse transcription file and records the audio ID - subtitle mapping.
    Returns False if the file could not be read.
    """

    file_directory = Path(file_path).parent
    file_name = Path(file_path).stem
    self.df.loc[len(self.df)] = {
        'id':file_name,
        'transcription':file_directory.name,
        'path': f'{file_directory}/{file_name}.wav'
    }
    return True

  def keys(self):
    return iter(self.df['id'])

  def get(self, id: int):
    """
    Retrieve a dataframe row from ID.
    """
    return self.df.loc[self.df['id'] == id]

In [24]:
import random

random.seed(42)

def split_dataframe(df, ratio):
    # Shuffle the DataFrame
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Calculate the split index
    split_index = int(len(df_shuffled) * ratio)

    # Split the DataFrame
    #df_1 = df_shuffled.iloc[:split_index]
    df_2 = df_shuffled.iloc[split_index:]

    return df_2.iloc[:int(len(df_2)*0.2)]

In [25]:
from tqdm import tqdm

import tqdm

from pathlib import Path


df_dict = dict()

all_subdirectories = glob.glob(
        f"/content/drive/MyDrive/cool-boy/datasets/single_words/*",
        recursive=True
)

for i in tqdm.tqdm(range(len(all_subdirectories))):
  path = all_subdirectories[i]
  fruit_label = Path(path).name
  ds = AudioDataset()
  if ds.load_transcriptions(path):
    df_dict[fruit_label] = split_dataframe(ds.df, 0.8)

100%|██████████| 30/30 [01:10<00:00,  2.36s/it]


In [26]:
df_dict_combined = pd.concat(df_dict.values(), ignore_index=True)
df_dict_combined = df_dict_combined.drop(columns=['id'])
df_dict_combined

,path,transcription
0,/content/drive/MyDrive/cool-boy/datasets/singl...,bed
1,/content/drive/MyDrive/cool-boy/datasets/singl...,bed
2,/content/drive/MyDrive/cool-boy/datasets/singl...,bed
3,/content/drive/MyDrive/cool-boy/datasets/singl...,bed
4,/content/drive/MyDrive/cool-boy/datasets/singl...,bed
...,...,...
1403,/content/drive/MyDrive/cool-boy/datasets/singl...,zero
1404,/content/drive/MyDrive/cool-boy/datasets/singl...,zero
1405,/content/drive/MyDrive/cool-boy/datasets/singl...,zero
1406,/content/drive/MyDrive/cool-boy/datasets/singl...,zero


In [27]:
audio_list = []
sampling_freq_list = []

for index, row in df_dict_combined.iterrows():
  audio, sampling_freq = librosa.load(row['path'], sr=16_000)

  audio_list.append(audio)
  sampling_freq_list.append(sampling_freq)

df_dict_combined['audio'] = audio_list
df_dict_combined['sampling_freq'] = sampling_freq_list

In [28]:
custom_dataset = Dataset.from_pandas(df_dict_combined)
custom_dataset

Dataset({
    features: ['path', 'transcription', 'audio', 'sampling_freq'],
    num_rows: 1408
})

In [29]:
custom_dataset = custom_dataset.train_test_split(test_size=0.2)
custom_dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'transcription', 'audio', 'sampling_freq'],
        num_rows: 1126
    })
    test: Dataset({
        features: ['path', 'transcription', 'audio', 'sampling_freq'],
        num_rows: 282
    })
})

## Loud our pretrained model

In [33]:
processor = AutoProcessor.from_pretrained("/content/drive/MyDrive/model/data2vec-model/checkpoint-2000", local_files_only=True)


model = AutoModelForCTC.from_pretrained("/content/drive/MyDrive/model/data2vec-model/checkpoint-2000")


def apply_processor(batch):
  batch = processor(batch["audio"], sampling_rate=batch["sampling_freq"], text=batch["transcription"])
  batch["input_length"] = len(batch["input_values"][0])
  return batch

encoded_datasets = custom_dataset.map(apply_processor, remove_columns=custom_dataset.column_names["train"], num_proc=4)



Map (num_proc=4):   0%|          | 0/1126 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/282 [00:00<?, ? examples/s]

In [34]:

# Need to create a data collator to prepare batches of data suitable for training CTC loss-based models
# Pad text and labels to length of the longest element in its batch to make it uniform length

@dataclass
class DataCollatorCTCLossWithPadding:
  processor: AutoProcessor
  padding: Union[bool, str] = "longest" # pad to the longest sequence

  def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

    #audio features
    input_features = [{"input_values": feature["input_values"][0]} for feature in features]
    # tokenized labels
    label_features = [{"input_ids": feature["labels"]} for feature in features]

    batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

    labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

    # replace padding with -100 to ignore loss correctly
    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

    batch["labels"] = labels

    return batch

data_collator = DataCollatorCTCLossWithPadding(processor=processor, padding="longest")

## Fine-Tune and Evaluate

In [35]:
wer_metric = evaluate.load("wer")

In [36]:


def compute_metrics(pred):
    # Compute predicted labels
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = processor.batch_decode(pred_ids)

    # Replace -100 with pad token ID in true labels
    true_labels = np.where(pred.label_ids == -100, processor.tokenizer.pad_token_id, pred.label_ids)
    true_str = processor.batch_decode(true_labels, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=true_str)

    return {"wer": wer}


In [38]:
training_args = TrainingArguments(
    output_dir="data2vec-benchmark",  # Output directory to save model checkpoints and logs
    group_by_length=True,  # Group samples of roughly the same length together to minimize padding
    per_device_train_batch_size=4,  # Batch size per GPU/device during training
    evaluation_strategy="steps",  # Evaluate every `eval_steps`
    fp16=True,  # Use mixed precision training with automatic mixed precision scaler
    save_steps=100,  # Save model checkpoint every `save_steps`
    eval_steps=100,  # Evaluate model every `eval_steps`
    logging_steps=50,  # Log training metrics every `logging_steps`
    learning_rate=5e-5,  # Learning rate
    weight_decay=0.001,  # Weight decay to prevent overfitting
    warmup_steps=100,  # Warmup steps for learning rate scheduler
    save_total_limit=1,  # Limit the total number of saved checkpoints
    max_steps=2000,  # Maximum number of training steps
)

# Define Trainer
trainer = Trainer(
    model=model,  # Model to be trained
    args=training_args,  # Training arguments
    train_dataset=encoded_datasets["train"],  # Training dataset
    eval_dataset=encoded_datasets["test"],  # Evaluation dataset
    tokenizer=processor,  # Tokenizer for preprocessing inputs
    data_collator=data_collator,  # Data collator for batching and padding
    compute_metrics=compute_metrics,  # Function to compute evaluation metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,2.919500,0.921046,1.000000
200,1.834600,0.700229,1.000000
300,2.300300,1.160645,1.000000
400,1.289400,1.813363,1.000000
500,0.712800,2.349385,1.000000
600,0.793100,1.464797,1.000000
700,0.870200,1.189365,1.000000
800,0.765100,1.184213,1.000000
900,0.580300,1.487588,1.000000
1000,0.985900,1.154379,1.000000


TrainOutput(global_step=2000, training_loss=1.3386932640075684, metrics={'train_runtime': 514.9377, 'train_samples_per_second': 15.536, 'train_steps_per_second': 3.884, 'total_flos': 7.1443826251776e+16, 'train_loss': 1.3386932640075684, 'epoch': 7.09})